In [0]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow2/semantic_parsing/tree_slu/main')

In [0]:
!pip install tensorflow-gpu==2.0.0-alpha0

    100% |████████████████████████████████| 332.1MB 60kB/s 
    100% |████████████████████████████████| 61kB 25.7MB/s 
    100% |████████████████████████████████| 419kB 13.6MB/s 
    100% |████████████████████████████████| 3.0MB 7.0MB/s 


In [0]:
import tensorflow as tf
import numpy as np
import pprint
import logging
import time
import nltk
import os

from pathlib import Path

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 2.0.0-alpha0
GPU Enabled: True


In [0]:
# stream data from text files
def data_generator(f_path, params):
  with open(f_path) as f:
    print('Reading', f_path)
    for line in f:
      text_raw, text_tokenized, label = line.split('\t')
      text_tokenized = text_tokenized.lower().split()
      label = label.replace('[', '[ ').lower().split()
      source = [params['tgt2idx'].get(w, len(params['tgt2idx'])) for w in text_tokenized]
      target = [params['tgt2idx'].get(w, len(params['tgt2idx'])) for w in label]
      target_in = [1] + target
      target_out = target + [2]
      yield (source, target_in, target_out)

In [0]:
def dataset(is_training, params):
  _shapes = ([None], [None], [None])
  _types = (tf.int32, tf.int32, tf.int32)
  _pads = (0, 0, 0)
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['train_samples'])
    ds = ds.padded_batch(params['train_batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['eval_batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
  return ds

In [0]:
def get_timing_signal_1d(length,
                         channels,
                         min_timescale=1.0,
                         max_timescale=1.0e4,
                         start_index=0):
  to_float = lambda x: tf.cast(x, tf.float32)
  position = to_float(tf.range(length) + start_index)
  num_timescales = channels // 2
  log_timescale_increment = (
      tf.math.log(float(max_timescale) / float(min_timescale)) /
      tf.maximum(to_float(num_timescales) - 1, 1))
  inv_timescales = min_timescale * tf.exp(
      to_float(tf.range(num_timescales)) * -log_timescale_increment)
  scaled_time = tf.expand_dims(position, 1) * tf.expand_dims(inv_timescales, 0)
  signal = tf.concat([tf.sin(scaled_time), tf.cos(scaled_time)], axis=1)
  signal = tf.pad(signal, [[0, 0], [0, tf.mod(channels, 2)]])
  signal = tf.reshape(signal, [1, length, channels])
  return signal

In [0]:
class LayerNorm(tf.keras.layers.Layer):
  def __init__(self, params):
    super().__init__()
    self._epsilon = params['epsilon']
    self._hidden_units = params['global_units']
  
  def build(self, input_shape):
    self.scale = self.add_weight(name='scale',
                                 shape=[self._hidden_units],
                                 initializer=tf.ones_initializer(),
                                 trainable=True)
    self.bias = self.add_weight(name='bias',
                                shape=[self._hidden_units],
                                initializer=tf.zeros_initializer(),
                                trainable=True)
    super().build(input_shape)
  
  def call(self, inputs):
    mean, variance = tf.nn.moments(inputs, [-1], keepdims=True)
    norm_x = (inputs - mean) * tf.math.rsqrt(variance + self._epsilon)
    return norm_x * self.scale + self.bias
  
  def compute_output_shape(self, input_shape):
    return input_shape

In [0]:
class PointwiseFFNBlock(tf.keras.Model):
  def __init__(self, params):
    super().__init__()
    self.layer_norm = LayerNorm(params)
    self.block_dropout = tf.keras.layers.Dropout(params['dropout_rate'])

    self.filter = tf.keras.layers.Dense(params['multiplier']*params['global_units'], tf.nn.relu, name='filter')
    self.dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    self.linear = tf.keras.layers.Dense(params['global_units'], name='linear')
  
  
  def call(self, inputs, training=False):
    x = self.layer_norm(inputs)
    x = self.forward(x, training=training)
    x = self.block_dropout(x, training=training)
    x += inputs
    return x
    
  
  def forward(self, x, training):
    return self.linear(self.dropout(self.filter(x), training=training))

In [0]:
class SelfAttentionBlock(tf.keras.Model):
  def __init__(self, params, is_bidirectional):
    super().__init__()
    
    self.layer_norm = LayerNorm(params)
    self.block_dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    
    self.qkv_linear = tf.keras.layers.Dense(3 * params['global_units'], name='qkv_linear')
    self.dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    self.out_linear = tf.keras.layers.Dense(params['global_units'], name='out_linear')
    
    self._is_bidirectional = is_bidirectional
    self._num_heads = params['num_heads']
  
  
  def call(self, inputs, training=False):
    inputs, masks = inputs
    x = self.layer_norm(inputs)
    x = self.forward((x, masks), training=training)
    x = self.block_dropout(x, training=training)
    x += inputs
    return x
    
  
  def forward(self, inputs, training):
    x, masks = inputs
    timesteps = tf.shape(x)[1]
    
    q_k_v = self.qkv_linear(x)
    q, k, v = tf.split(q_k_v, 3, axis=-1)
    
    if self._num_heads > 1:
      q = tf.concat(tf.split(q, self._num_heads, axis=2), axis=0)                        
      k = tf.concat(tf.split(k, self._num_heads, axis=2), axis=0)                        
      v = tf.concat(tf.split(v, self._num_heads, axis=2), axis=0)
    
    align = tf.matmul(q, k, transpose_b=True)
    align *= tf.math.rsqrt(tf.cast(k.shape[-1], tf.float32))
    
    if (masks is not None) or (not self._is_bidirectional):
      paddings = tf.fill(tf.shape(align), float('-inf'))
    
    if masks is not None:
      c_masks = tf.tile(masks, [params['num_heads'], 1])
      c_masks = tf.tile(tf.expand_dims(c_masks, 1), [1, timesteps, 1])
      align = tf.where(tf.equal(c_masks, 0), paddings, align)
    
    if not self._is_bidirectional:
      lower_tri = tf.ones((timesteps, timesteps))                                       
      lower_tri = tf.linalg.LinearOperatorLowerTriangular(lower_tri).to_dense()      
      t_masks = tf.tile(tf.expand_dims(lower_tri, 0), [tf.shape(align)[0], 1, 1])     
      align = tf.where(tf.equal(t_masks, 0), paddings, align)
    
    align = tf.nn.softmax(align)
    align = self.dropout(align, training=training)
    
    if masks is not None:
      q_masks = tf.tile(masks, [params['num_heads'], 1])
      q_masks = tf.tile(tf.expand_dims(q_masks, 2), [1, 1, timesteps])
      align *= tf.cast(q_masks, tf.float32)
    
    x = tf.matmul(align, v)
    if self._num_heads > 1:
      x = tf.concat(tf.split(x, self._num_heads, axis=0), axis=2)
    x = self.out_linear(x)
    
    return x

In [0]:
class MutualAttentionBlock(tf.keras.Model):
  def __init__(self, params):
    super().__init__()
    self.layer_norm = LayerNorm(params)
    self.block_dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    
    self.q_linear = tf.keras.layers.Dense(params['global_units'], name='q_linear')
    self.kv_linear = tf.keras.layers.Dense(2*params['global_units'], name='kv_linear')
    self.dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    self.out_linear = tf.keras.layers.Dense(params['global_units'], name='out_linear')
    
    self._num_heads = params['num_heads']
  
  
  def call(self, inputs, training=False):
    inputs, mask_dec, encoded, mask_enc = inputs
    x = self.layer_norm(inputs)
    x = self.forward((x, mask_dec, encoded, mask_enc), training=training)
    x = self.block_dropout(x, training=training)
    x += inputs
    return x
    
  
  def forward(self, inputs, training):
    query, mask_query, context, mask_context = inputs
    time_query, time_context = tf.shape(query)[1], tf.shape(context)[1]
    
    q = self.q_linear(query)
    k_v = self.kv_linear(context)
    k, v = tf.split(k_v, 2, axis=-1)
    
    if self._num_heads > 1:
      q = tf.concat(tf.split(q, self._num_heads, axis=2), axis=0)                        
      k = tf.concat(tf.split(k, self._num_heads, axis=2), axis=0)                        
      v = tf.concat(tf.split(v, self._num_heads, axis=2), axis=0)
    
    align = tf.matmul(q, k, transpose_b=True)
    align *= tf.math.rsqrt(tf.cast(k.shape[-1], tf.float32))
    
    paddings = tf.fill(tf.shape(align), float('-inf'))
    context_masks = tf.tile(mask_context, [self._num_heads, 1])
    context_masks = tf.tile(tf.expand_dims(context_masks, 1), [1, time_query, 1])
    align = tf.where(tf.equal(context_masks, 0), paddings, align)
    
    align = tf.nn.softmax(align)
    align = self.dropout(align, training=training)
    
    query_masks = tf.tile(mask_query, [self._num_heads, 1])
    query_masks = tf.tile(tf.expand_dims(query_masks, 2), [1, 1, time_context])
    align *= tf.cast(query_masks, tf.float32)
    
    x = tf.matmul(align, v)
    if self._num_heads > 1:
      x = tf.concat(tf.split(x, self._num_heads, axis=0), axis=2)
    x = self.out_linear(x)
    
    return x

In [0]:
class EncoderLayer(tf.keras.Model):
  def __init__(self, params, name):
    super().__init__(name=name)
    self.self_attention = SelfAttentionBlock(params, is_bidirectional=True)
    self.pointwise_ffn = PointwiseFFNBlock(params)
  
  
  def call(self, inputs, training=False):
    x, mask = inputs
    x = self.self_attention((x, mask), training=training)
    x = self.pointwise_ffn(x, training=training)
    return x

In [0]:
class DecoderLayer(tf.keras.Model):
  def __init__(self, params, name):
    super().__init__(name=name)
    self.self_attention = SelfAttentionBlock(params, is_bidirectional=False)
    self.mutual_attention = MutualAttentionBlock(params)
    self.pointwise_ffn = PointwiseFFNBlock(params)
  
  
  def call(self, inputs, training=False):
    decoded, mask_dec, encoded, mask_enc = inputs
    decoded = self.self_attention((decoded, mask_dec), training=training)
    decoded = self.mutual_attention((decoded, mask_dec, encoded, mask_enc), training=training)
    decoded = self.pointwise_ffn(decoded, training=training)
    return decoded

In [0]:
class Model(tf.keras.Model):
  def __init__(self, params):
    super().__init__()
    self.params = params
    self.embedding = tf.keras.layers.Embedding(len(params['tgt2idx']), params['global_units'],
      embeddings_initializer=tf.initializers.RandomNormal(stddev=params['global_units'] ** -0.5))
    self.input_dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    self.encoder_layers = [EncoderLayer(params, name='encoder_layer_{}'.format(i+1)) for i in range(params['num_layers'])]
    self.decoder_layers = [DecoderLayer(params, name='decoder_layer_{}'.format(i+1)) for i in range(params['num_layers'])]
    self.out_bias = self.add_weight(name='out_b', shape=[len(params['tgt2idx'])])
    
  
  def call(self, inputs, training=False):
    input_enc, input_dec = inputs
    
    mask_enc = tf.sign(input_enc)
    mask_dec = tf.sign(input_dec)
    
    input_enc = self.embedding(input_enc)
    input_dec = self.embedding(input_dec)
    
    encoded = self.input_transform(input_enc, training=training)
    decoded = self.input_transform(input_dec, training=training)
    
    for layer in self.encoder_layers:
      encoded = layer((encoded, mask_enc), training=training)
    for layer in self.decoder_layers:
      decoded = layer((decoded, mask_dec, encoded, mask_enc), training=training)
    
    axis_1, axis_2  = tf.shape(decoded)[0], tf.shape(decoded)[1]
    decoded = tf.reshape(decoded, (axis_1*axis_2, params['global_units']))
    logits = tf.matmul(decoded, self.embedding.embeddings, transpose_b=True)
    logits = tf.reshape(logits, (axis_1, axis_2, len(self.params['tgt2idx'])))
    logits = tf.nn.bias_add(logits, self.out_bias)
    return logits
  
  
  def input_transform(self, x, training):
    if self.params['is_embedding_scaled']:
      x *= tf.sqrt(tf.cast(self.params['global_units'], tf.float32))
    x += get_timing_signal_1d(tf.shape(x)[1], self.params['global_units'])
    x = self.input_dropout(x, training=training)
    return x

In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip()
      word2idx[line] = i
  return word2idx

In [0]:
def is_descending(history: list) -> bool:
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True

In [0]:
params = {
    'train_path': '../data/train.tsv',
    'test_path': '../data/test.tsv',
    'vocab_src_path': '../vocab/source.txt',
    'vocab_tgt_path': '../vocab/target.txt',
    'model_path': '../model',
    'num_layers': 3,
    'dropout_rate': 0.1,
    'global_units': 256,
    'num_heads': 4,
    'multiplier': 2,
    'epsilon': 1e-6,
    'is_embedding_scaled': True,
    'max_decode_len': 50,
    'lr': 4e-4,
    'train_samples': 31279,
    'train_batch_size': 32,
    'eval_samples': 9042,
    'eval_batch_size': 300,
    'num_patience': 5,
}

In [0]:
params['tgt2idx'] = get_vocab(params['vocab_tgt_path'])
params['idx2tgt'] = {idx: tgt for tgt, idx in params['tgt2idx'].items()}

In [0]:
Path(params['model_path']).mkdir(exist_ok=True)
model = Model(params)
model.build([[None, None], [None, None]])
pprint.pprint([(v.name, v.shape) for v in model.trainable_variables])

[('embedding/embeddings:0', TensorShape([8691, 256])),
 ('encoder_layer_1/self_attention_block/layer_norm/scale:0',
  TensorShape([256])),
 ('encoder_layer_1/self_attention_block/layer_norm/bias:0', TensorShape([256])),
 ('encoder_layer_1/self_attention_block/qkv_linear/kernel:0',
  TensorShape([256, 768])),
 ('encoder_layer_1/self_attention_block/qkv_linear/bias:0', TensorShape([768])),
 ('encoder_layer_1/self_attention_block/out_linear/kernel:0',
  TensorShape([256, 256])),
 ('encoder_layer_1/self_attention_block/out_linear/bias:0', TensorShape([256])),
 ('encoder_layer_1/pointwise_ffn_block/layer_norm_1/scale:0',
  TensorShape([256])),
 ('encoder_layer_1/pointwise_ffn_block/layer_norm_1/bias:0',
  TensorShape([256])),
 ('encoder_layer_1/pointwise_ffn_block/filter/kernel:0',
  TensorShape([256, 512])),
 ('encoder_layer_1/pointwise_ffn_block/filter/bias:0', TensorShape([512])),
 ('encoder_layer_1/pointwise_ffn_block/linear/kernel:0',
  TensorShape([512, 256])),
 ('encoder_layer_1/poin

In [0]:
decay_lr = tf.optimizers.schedules.ExponentialDecay(params['lr'], 1000, 0.99)
optim = tf.optimizers.Adam(params['lr'])
global_step = 0

In [0]:
history_acc = []
best_acc = .0

In [0]:
t0 = time.time()
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)

In [0]:
def minimal_test(model, params):
  test_str = ['what', 'times', 'are', 'the', 'nutcracker', 'show', 'playing', 'near', 'me']
  test_arr = tf.convert_to_tensor([[params['tgt2idx'][w] for w in test_str]])
  generated = tf.convert_to_tensor([[1]])
  for i in range(params['max_decode_len']):
    logits = model((test_arr, generated), training=False)
    ids = tf.argmax(logits[:, i, :], axis=-1, output_type=tf.int32)
    ids = tf.expand_dims(ids, 1)
    generated = tf.concat((generated, ids), axis=1)
    if np.asscalar(ids.numpy()) == 2:
      break
  print('-'*12)
  print('minimal test')
  print('utterance:', ' '.join(test_str))
  parsed = ' '.join([params['idx2tgt'][idx] for idx in generated[0].numpy()[1:-1]])
  print('parsed:', parsed)
  print()
  try:
    nltk.tree.Tree.fromstring(parsed.replace('[ ', '(').replace(' ]', ')')).pretty_print()
  except:
    pass
  print('-'*12)

In [0]:
def is_descending(history: list) -> bool:
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True  

In [0]:
while True:
  # TRAINING
  for (source, target_in, target_out) in dataset(is_training=True, params=params):
    with tf.GradientTape() as tape:
      logits = model((source, target_in), training=True)
      loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=target_out, logits=logits)
      weights = tf.cast(tf.sign(target_out), tf.float32)
      loss = tf.reduce_sum(loss * weights) / tf.reduce_sum(weights)
      
    optim.lr.assign(decay_lr(global_step))
    grads = tape.gradient(loss, model.trainable_variables)
    optim.apply_gradients(zip(grads, model.trainable_variables))
    
    if global_step % 50 == 0:
      logger.info("Step {} | Loss: {:.4f} | Spent: {:.1f} secs | LR: {:.6f}".format(
          global_step, loss.numpy().item(), time.time()-t0, optim.lr.numpy().item()))
      t0 = time.time()
    
    global_step += 1
  
  # EVALUATION
  minimal_test(model, params)
  m = tf.keras.metrics.Mean()
  
  for i, (source, target_in, target_out) in enumerate(dataset(is_training=False, params=params)):
    generated = tf.ones((source.shape[0], 1), tf.int32)
    for j in range(target_out.shape[1]):
      logits = model((source, generated), training=False)
      ids = tf.argmax(logits[:, j, :], axis=-1, output_type=tf.int32)
      ids = tf.expand_dims(ids, 1)
      generated = tf.concat((generated, ids), axis=1)

    seq_lens = tf.argmax(tf.cast(tf.equal(target_out, 2), tf.int32), axis=1)
    for pred, tgt, seq_len in zip(generated.numpy(), target_out.numpy(), seq_lens.numpy()):
      pred = pred[1:][:seq_len+1]
      tgt = tgt[:seq_len+1]
      matched = np.all(pred == tgt)
      m.update_state(int(matched))
    print("Testing [{}/{}]".format(i, params['eval_samples']//params['eval_batch_size']))
  
  acc = m.result().numpy()
  logger.info("Evaluation: Testing Exact Match Accuracy: {:.3f}".format(acc))
  history_acc.append(acc)

  if acc > best_acc:
    best_acc = acc
    model.save_weights(os.path.join(params['model_path'], '{}'.format(global_step)))
  logger.info("Best Accuracy: {:.3f}".format(best_acc))

  if len(history_acc) > params['num_patience'] and is_descending(history_acc):
    logger.info("Testing Accuracy not improved over {} epochs, Early Stop".format(params['num_patience']))
    break

W0429 00:29:04.201907 140543929796480 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:410: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


Reading ../data/train.tsv


I0429 00:29:11.423241 140543929796480 interactiveshell.py:2882] Step 0 | Loss: 15.3380 | Spent: 7.3 secs | LR: 0.000400
I0429 00:29:39.555497 140543929796480 interactiveshell.py:2882] Step 50 | Loss: 4.1791 | Spent: 28.1 secs | LR: 0.000400
I0429 00:30:06.763456 140543929796480 interactiveshell.py:2882] Step 100 | Loss: 2.6062 | Spent: 27.2 secs | LR: 0.000400
I0429 00:30:35.062666 140543929796480 interactiveshell.py:2882] Step 150 | Loss: 2.2839 | Spent: 28.3 secs | LR: 0.000399
I0429 00:31:02.228696 140543929796480 interactiveshell.py:2882] Step 200 | Loss: 1.5065 | Spent: 27.2 secs | LR: 0.000399
I0429 00:31:31.130009 140543929796480 interactiveshell.py:2882] Step 250 | Loss: 1.7638 | Spent: 28.9 secs | LR: 0.000399
I0429 00:31:59.445777 140543929796480 interactiveshell.py:2882] Step 300 | Loss: 1.4636 | Spent: 28.3 secs | LR: 0.000399
I0429 00:32:26.583513 140543929796480 interactiveshell.py:2882] Step 350 | Loss: 1.2231 | Spent: 27.1 secs | LR: 0.000399
I0429 00:32:53.779572 14054

------------
minimal test
utterance: what times are the nutcracker show playing near me
parsed: [ in:get_event what times are the nutcracker playing ]

           in:get_event                       
  ______________|_________________________     
what times     are      the nutcracker playing

------------
Reading ../data/test.tsv
Testing [0/30]
Testing [1/30]
Testing [2/30]
Testing [3/30]
Testing [4/30]
Testing [5/30]
Testing [6/30]
Testing [7/30]
Testing [8/30]
Testing [9/30]
Testing [10/30]
Testing [11/30]
Testing [12/30]
Testing [13/30]
Testing [14/30]
Testing [15/30]
Testing [16/30]
Testing [17/30]
Testing [18/30]
Testing [19/30]
Testing [20/30]
Testing [21/30]
Testing [22/30]
Testing [23/30]
Testing [24/30]
Testing [25/30]
Testing [26/30]
Testing [27/30]
Testing [28/30]
Testing [29/30]


I0429 00:54:26.871132 140543929796480 interactiveshell.py:2882] Evaluation: Testing Exact Match Accuracy: 0.299
I0429 00:54:27.015942 140543929796480 interactiveshell.py:2882] Best Accuracy: 0.299


...

------------
minimal test
utterance: what times are the nutcracker show playing near me
parsed: [ in:get_event what times are [ sl:category_event the nutcracker show ] playing [ sl:location [ in:get_location [ sl:search_radius near ] [ sl:location_user me ] ] ] ]

                       in:get_event                                                                        
  __________________________|_____________________________________________________                          
 |     |    |     |                        |                                 sl:location                   
 |     |    |     |                        |                                      |                         
 |     |    |     |                        |                               in:get_location                 
 |     |    |     |                        |                      ________________|_______________          
 |     |    |     |                 sl:category_even      sl:search_radius          

I0429 09:54:01.033562 140543929796480 interactiveshell.py:2882] Evaluation: Testing Exact Match Accuracy: 0.683
I0429 09:54:01.034597 140543929796480 interactiveshell.py:2882] Best Accuracy: 0.688


Testing [30/30]
Reading ../data/train.tsv


I0429 09:54:26.128770 140543929796480 interactiveshell.py:2882] Step 21550 | Loss: 0.0274 | Spent: 1044.5 secs | LR: 0.000322


KeyboardInterrupt: ignored